In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, ensemble, tree, multioutput, preprocessing, model_selection, metrics
import xgboost as xgb

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
visiteurs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seo-visiteur.csv')
visiteurs_journey = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seo_visits.csv')

In [ ]:
visiteurs['is_bot'] = False
visiteurs['user_id'] = visiteurs['id']

In [ ]:
visiteurs = visiteurs.drop(columns=[
                        'country_name', 'continent_code',	'continent_name', 'session_id','first_visit_date', 'id',
                        'dma_code',	'is_in_european_union',	'latitude'	,'longitude'	,'city',	'region',	'time_zone',	'postal_code'
                        ]
               )

In [ ]:
visiteurs['HTTP_USER_AGENT'].isna().sum()

1

In [ ]:
visiteurs['HTTP_USER_AGENT'] = visiteurs['HTTP_USER_AGENT'].fillna('bot')

In [ ]:
visiteurs

,ip,HTTP_USER_AGENT,country_code,is_bot,user_id
0,190.148.252.233,Mozilla/5.0 (Linux; Android 8.1.0; SM-J410G Bu...,GT,False,8149
1,189.176.147.206,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,MX,False,8148
2,189.176.121.235,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,MX,False,8147
3,201.141.106.211,Mozilla/5.0 (Linux; Android 11; SM-A307G) Appl...,MX,False,8146
4,179.51.3.203,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SV,False,8145
...,...,...,...,...,...
8144,201.190.251.223,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,AR,False,5
8145,213.195.109.213,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6...,ES,False,4
8146,187.207.19.1,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,MX,False,3
8147,187.162.27.37,Mozilla/5.0 (iPhone; CPU iPhone OS 15_3_1 like...,MX,False,2


In [ ]:
visiteurs.loc[
              visiteurs[
                        'HTTP_USER_AGENT'
                        ].str.contains(
                            '|'.join(
                                [
                                 'python','java', 'go', 'twitter', 'facebook','requests', 'bot'
                                 ]
                                )
                            ), 'is_bot'
                            ] = True

In [ ]:
visiteurs_journey = visiteurs_journey.drop(columns=[
                        'id', 'parsed'
                        ]
               )
visiteurs_journey

,date,current_path,comes_from,user_id
0,2022-03-09 02:32:07.129352+00,https://inversionesyfinanzas.xyz/publicaciones...,https://www.google.com/,1
1,2022-03-09 03:34:44.020298+00,https://inversionesyfinanzas.xyz/publicaciones...,https://www.google.com.mx/,2
2,2022-03-09 03:35:32.797788+00,https://inversionesyfinanzas.xyz/publicaciones...,https://www.google.com.mx/,2
3,2022-03-09 08:17:12.909467+00,https://inversionesyfinanzas.xyz/definicion/ca...,https://www.google.com/,3
4,2022-03-09 09:57:21.575207+00,https://inversionesyfinanzas.xyz/favicon.ico,https://lucas.inversionesyfinanzas.xyz/static/...,4
...,...,...,...,...
148555,2022-06-05 21:39:33.809669+00,https://inversionesyfinanzas.xyz/questions?pag...,None,1991
148556,2022-06-05 21:40:29.227965+00,https://inversionesyfinanzas.xyz/screener/anal...,None,30
148557,2022-06-05 21:48:50.763618+00,https://inversionesyfinanzas.xyz/,http://198.58.97.139:80/,7043
148558,2022-06-05 21:51:14.886249+00,https://inversionesyfinanzas.xyz/screener/anal...,None,43


In [ ]:
total_seo = pd.merge(visiteurs_journey, visiteurs, on=["user_id"])

In [ ]:
total_seo['date'] = pd.to_datetime(total_seo['date'])

In [ ]:
categorical_cols = total_seo.select_dtypes('object').columns.tolist()
categorical_cols

['current_path', 'comes_from', 'ip', 'HTTP_USER_AGENT', 'country_code']

In [ ]:
category_array = total_seo[categorical_cols[4]].values.reshape(1, -1)

In [ ]:
ohe = preprocessing.OneHotEncoder(categories='auto')
X = ohe.fit_transform(category_array).toarray()

In [ ]:
X

array([[1., 1., 1., ..., 1., 1., 1.]])

In [ ]:
encoder = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore').fit(category_array)
encoded_cols = list(encoder.get_feature_names_out(total_seo['country_code']))
total_seo[encoded_cols] = encoder.transform(X)
total_seo = total_seo.drop(columns=['country_code'])

TypeError: ignored

In [ ]:
cv = CountVectorizer()
features = cv.fit_transform(total_seo['HTTP_USER_AGENT'])

In [ ]:
features